In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path

In [6]:
dataset_path = Path(f'../datasets/drone_2/') 
images_path = dataset_path / "images"
mapping_path = images_path / "mapping"
query_path = images_path / "query"

In [50]:
queries = [p.relative_to(images_path).as_posix() for p in (query_path).iterdir()]

# parse image names to collect altitudes and create folders
altitudes = []
DJI_intrinsics = np.eye(3)

# fov = 82.1
# fx = (width * 0.5) / np.tan(fov * 0.5*(np.pi/180))
# fy = (height * 0.5) / np.tan(fov * 0.5*(np.pi/180))
# cx = width * 0.5
# cy = height * 0.5

DJI_intrinsics[0,0], DJI_intrinsics[1,1], DJI_intrinsics[0,2], DJI_intrinsics[1,2] = 2204.82, 2204.82, 1080, 1920
inv_I = np.linalg.inv(DJI_intrinsics)
print(inv_I)

for q in queries:
    altitude = int(q.split("=")[-1].split("_")[0].split(".")[0])
    altitude_path = mapping_path / f"altitude_{altitude}"
    
    img = cv2.imread(str(images_path / q))
    height, width, c = img.shape
    
    corners = np.array([np.array([0,0]), 
                        np.array([height, 0]), 
                        np.array([0, width]), 
                        np.array([height, width])])
    
    point3d = np.ones((4,3))
    
    for i, c in enumerate(corners):
        p3d = point3d[i, :]
        p3d[:2] = c
        p3d = (inv_I @ p3d.T) * -altitude
        point3d[i] = p3d
    
    width_m = point3d[0, 1] - point3d[2, 1]
    heigth_m = point3d[0, 0] - point3d[1, 0]
    
    w_px_ratio = width / width_m
    h_px_ratio = height / heigth_m
    #print(width_m, heigth_m, w_px_ratio, h_px_ratio)
    
    alt_struct = {"altitude": altitude, "path": altitude_path, "pixel_ratio": w_px_ratio}
    
    if alt_struct not in altitudes:
        print(f"Creating a dir for altitude {alt_struct['altitude']}m...")
        alt_struct["path"].mkdir(parents=True, exist_ok=True)
        altitudes.append(alt_struct)

print(altitudes)

[[ 4.53551764e-04  0.00000000e+00 -4.89835905e-01]
 [ 0.00000000e+00  4.53551764e-04 -8.70819387e-01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Creating a dir for altitude 2m...
Creating a dir for altitude 3m...
Creating a dir for altitude 4m...
[{'altitude': 2, 'path': PosixPath('../datasets/drone_2/images/mapping/altitude_2'), 'pixel_ratio': 1102.41}, {'altitude': 3, 'path': PosixPath('../datasets/drone_2/images/mapping/altitude_3'), 'pixel_ratio': 734.94}, {'altitude': 4, 'path': PosixPath('../datasets/drone_2/images/mapping/altitude_4'), 'pixel_ratio': 551.205}]


In [60]:
# Resize gallery images with the corresponding altitudes
gallery = [p.relative_to(images_path).as_posix() for p in (mapping_path).iterdir() if p.is_file()]

I = np.eye(3)
I[0,0], I[1,1], I[0,2], I[1,2] = 1420.1306, 1420.1306, 964, 720
inv_I = np.linalg.inv(I)

print(gallery)
for a in altitudes:
    for img_path in gallery:
        img = cv2.imread(str(images_path / img_path))
        img_name = img_path.split("/")[-1]
        print(img_name)
        
        
        height, width, c = img.shape
        
        corners = np.array([np.array([0,0]), 
                            np.array([height, 0]), 
                            np.array([0, width]), 
                            np.array([height, width])])
    
        point3d = np.ones((4,3))
        
        altitude = 1
        
        for i, c in enumerate(corners):
            p3d = point3d[i, :]
            p3d[:2] = c
            p3d = (inv_I @ p3d.T) * -altitude
            point3d[i] = p3d
        
        
        width_m = point3d[0, 1] - point3d[2, 1]
        heigth_m = point3d[0, 0] - point3d[1, 0]
        
        px_ratio = a['pixel_ratio']
        
        new_px_width = int(width_m * px_ratio)
        new_px_heigth = int(heigth_m * px_ratio)
        
        resized_img = cv2.resize(img, (new_px_width, new_px_heigth), interpolation = cv2.INTER_AREA)
        cv2.imwrite(str(a["path"] / img_name), resized_img)
        
        print("new size: ", resized_img.shape)
        
        
        
    
    

['mapping/rgb_1920x1440_00000.jpeg', 'mapping/rgb_1920x1440_00001.jpeg', 'mapping/rgb_1920x1440_00002.jpeg', 'mapping/rgb_1920x1440_00003.jpeg', 'mapping/rgb_1920x1440_00004.jpeg', 'mapping/rgb_1920x1440_00005.jpeg', 'mapping/rgb_1920x1440_00006.jpeg', 'mapping/rgb_1920x1440_00007.jpeg', 'mapping/rgb_1920x1440_00008.jpeg', 'mapping/rgb_1920x1440_00009.jpeg', 'mapping/rgb_1920x1440_00010.jpeg', 'mapping/rgb_1920x1440_00011.jpeg']
rgb_1920x1440_00000.jpeg
new size:  (1117, 1490, 3)
rgb_1920x1440_00001.jpeg
new size:  (1117, 1490, 3)
rgb_1920x1440_00002.jpeg
new size:  (1117, 1490, 3)
rgb_1920x1440_00003.jpeg
new size:  (1117, 1490, 3)
rgb_1920x1440_00004.jpeg
new size:  (1117, 1490, 3)
rgb_1920x1440_00005.jpeg
new size:  (1117, 1490, 3)
rgb_1920x1440_00006.jpeg
new size:  (1117, 1490, 3)
rgb_1920x1440_00007.jpeg
new size:  (1117, 1490, 3)
rgb_1920x1440_00008.jpeg
new size:  (1117, 1490, 3)
rgb_1920x1440_00009.jpeg
new size:  (1117, 1490, 3)
rgb_1920x1440_00010.jpeg
new size:  (1117, 1490